In [23]:
import math
from fractions import Fraction

In [30]:
def round_inches(value, smallest_fraction=2):
    """
    Round to nearest 1/smallest_fraction inches.
    Example: smallest_fraction=2 -> nearest 1/2", 4 -> 1/4", 8 -> 1/8", etc.
    """
    rounded = round(value * smallest_fraction) / smallest_fraction
    return rounded

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

In [31]:
def find_Hs(W, U, alpha0, A0, alphaZ, Az, d_h, d_r, d_z): 
    """
    Calculates the primary beam scattered from the primary barrier.

    Parameters:
    W (float): Workload (Gy/wk) for the worst-case Wall G.
    U (float): Usage factor for the worst-case Wall G.
    alpha0 (float): Albedo of the primary barrier, dependent on material, energy, 
                    angle of incidence, and reflection.
    A0 (float): Dependent on field size and d_h (m^2).
    alphaZ (float): Albedo of Wall Z, dependent on material, energy, 
                    angle of incidence, and reflection.
    Az (float): Area of the inner maze entrance as viewed from the center of the 
                primary barrier projected onto Wall Z (m^2).
    d_h (float): Distance from the source to the proximal surface of the primary barrier (m).
    d_r (float): Distance from the center of the primary barrier, past the edge of IMW, 
                 to point B (m).
    d_z (float): Distance from the point B to the plane of the door (m).
    """

    H_s = (W * U * alpha0 * A0 * alphaZ * Az) / (d_h * d_r * d_z)**2
    return H_s


def find_Hps(W, U, a, F, alpha1, A1, d_sca, d_sec, d_zz): 
    """

    Parameters:
    
    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    a : scatter fraction dependent on scatter angle and beam energy 
    F : field size (cm^2)
    alpha1 : albedo of A1. Dependent on material, 0.5 MeV, angle of incidence, angle of reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sca : distance from source to iso (m)
    d_sec : distance from iso to center of A1 (m)
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ps = (W * U * a * (F/400)* alpha1 * A1) / (d_sca * d_sec * d_zz)**2
    return H_ps


def find_Hls(W, U, C, alpha_ls, A1, d_sec, d_zz):
    """
    
    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    C : IMRT factor (unitless)
    alpha_ls : albedo of A1, dependent on material, Co-60
               angle of incidence, and reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sec : distance from iso to center of A1 (m) if gantry rotates uniformly
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ls = (10**-3 *W * U * C * alpha_ls * A1) / (d_sec * d_zz)**2
    return H_ls


def find_Hlt(W, C, U, t_slant, TVL1, TVLe, d_L):
    """

    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    C : IMRT factor (unitless)
    U : usage factor for worst case Wall G
    t_slant : slant thickness of IMW (cm)
    TVL1 : TVL of IMW material (cm). Dependent on incident energy
    TVLe : equilibrium TVL of IMW material (cm). Dependent on incident energy
    d_L : distance from source to center of plane of door (m)
    """

    H_lt = (10**-3 * W * C * U * 10**-(1 + (t_slant - TVL1) / TVLe)) / d_L**2
    return H_lt

def verify_2barrier_vault2(P, T, thickness,d,tvl_6mv_ps,a_6mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    '''
  
    H_l_6mv = H_leak_unshielded(1435.5, d) * B2(thickness, 34, 29)

    H_ps_6mv = H_ps_unshielded(319, d, 1, a_6mv, 400) * B1(thickness, tvl_6mv_ps)

    H_total = H_ps_6mv + H_l_6mv 
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print(f"H_ps_6mv: {H_ps_6mv} mSv/wk")
    print(f"H_l_6mv: {H_l_6mv} mSv/wk")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")

# Important note: thickness must be in centimeters.

In [26]:
P = 1
U = 0.25 # use factor 
T = 1

# maze width
maze_width = convert_to_meters(8,5.00242)
# maze height
maze_height = convert_to_meters(10,0)
# inner maze height
inner_maze_height = convert_to_meters(9,0)
inner_maze_entrance = convert_to_meters(8,0)
# dz
dz = convert_to_meters(24,10.16134)
# dzz
dzz = convert_to_meters(34,3.0)

test1 = dz / (maze_width * maze_height)**0.5

test2 = dzz / (maze_width * maze_height)**0.5

if test1 < 6 and test1 > 2:
    print('PASS')
    print(test1)

if test2 < 6 and test2 > 2:
    print('PASS')
    print(test2)

if 1 < maze_height / maze_width and maze_height / maze_width < 2:
    print('PASS')
    print(maze_height/maze_width)

print(f"maze height = {maze_height} m")
print(f"maze width = {maze_width} m")
print(f"inner maze entrance = {inner_maze_entrance} m")
print(f"inner maze height = {inner_maze_height} m")
print(f"dz = {dz} m")
print(f"dzz = {dzz} m")


PASS
2.7082878336568914
PASS
3.7332348306221803
PASS
1.188090344765997
maze height = 3.048 m
maze width = 2.5654614679999996 m
inner maze entrance = 2.4384 m
inner maze height = 2.7432 m
dz = 7.573298036 m
dzz = 10.4394 m


In [ ]:
# H_s
# Variables for Vault #1
d_r = convert_to_meters(25,6.21549)
d_h = convert_to_meters(19,1) + 1
area_0 = (0.20 * d_h)**2 # m, Field size is 20 cm x 20 cm # CHECK THIS AREA
alpha_z = 13e-3 # Albedo of Wall Z, concrete, 0.5 MeV, 0 deg incidence, 60 deg reflection
alpha_0_6MV = 4e-3 # Albedo of Area 0, concrete, 6MV, 0 deg incidence, 60 deg reflection. Rounded down from 72 to 60

# Find A_z
width_Az = convert_to_meters(9,5.67737)
area_z = width_Az * inner_maze_height

print("H_s")
print("--------------------------")
print(f"dz = {dz} m")
print(f"dzz = {dzz} m")
print(f"d_r = {d_r} m")
print(f"d_h = {d_h} m")
print("--------------------------")
print(f"alpha_0_6MV = {alpha_0_6MV}")
print(f"alpha_z = {alpha_z}")
print("--------------------------")
print(f"width_Az = {width_Az} m")
print(f"area_0 = {area_0} m^2")
print(f"area_z = {area_z} m^2")

H_s_6MV = find_Hs(319, 0.25, alpha_0_6MV, area_0, alpha_z, area_z, d_h, d_r, dz)

print("---------------------------")
print(f"H_s_6MV = {H_s_6MV} Sv/wk")
print("--------------------------")


H_s
--------------------------
dz = 7.573298036 m
dzz = 10.4394 m
d_r = 7.777873446 m
d_h = 6.8166 m
--------------------------
alpha_0_6MV = 0.004
alpha_z = 0.013
--------------------------
width_Az = 2.887405198 m
area_0 = 1.8586414224000003 m^2
area_z = 7.9207299391536 m^2
---------------------------
H_s_6MV = 3.786756604162865e-07 Sv/wk
--------------------------


In [28]:
# Workload for H_ps is the same as H_s

# H_ps
d_sec = convert_to_meters(31,8.02)
d_sca = 1 # m
a_6MV = 1.39e-3 # 50 degree scatter angle. Rounded down from 50 to 45 deg
alpha_1_ps = 22e-3 # albedo of A1, concrete, 0.5 MeV, 45 deg incidence, 0 deg reflection
width_A1 = convert_to_meters(10,11.78769)
area_1 = maze_height * width_A1 # m^2

print("H_ps")
print("--------------------------")
print(f"d_zz = {dzz} m")
print(f"d_sec = {d_sec} m")
print(f"d_sca = {d_sca} m")
print("--------------------------")
print(f"a_6MV = {a_6MV}")
print(f"alpha_1_ps = {alpha_1_ps}")
print("---------------------------")
print(f"width_A1 = {width_A1} m")
print(f"area_1 = {area_1} m^2")

H_ps_6MV = find_Hps(319, 0.25, 400, a_6MV, alpha_1_ps, area_1, d_sca, d_sec, dzz)

print("---------------------------")
print(f"H_ps_6MV = {H_ps_6MV} Sv/wk")
print("---------------------------")



H_ps
--------------------------
d_zz = 10.4394 m
d_sec = 9.652508000000001 m
d_sca = 1 m
--------------------------
a_6MV = 0.00139
alpha_1_ps = 0.022
---------------------------
width_A1 = 3.347407326 m
area_1 = 10.202897529648 m^2
---------------------------
H_ps_6MV = 2.450531448182399e-06 Sv/wk
---------------------------


In [44]:
# P = 0.1 mSv/wk
# T = 1 
# Angle is 55.33 with the gantry pointing towards Wall C
# NCRP lists occupancy outside a door as 1/8, but we can be conservtative and use T=1
distance_test = convert_to_meters(25,6.17495) # m
thickness_IMW = convert_to_meters(2,11.82342) * 100 # convert to cm
test = thickness_IMW / math.cos(math.radians(55.33)) # corrected thickness for 23.54 degrees

verify_2barrier_vault2(0.1,1, thickness_IMW, distance_test, 23, 1.39e-3)

PASS
H_ps_6mv: 0.0008110490853974909 mSv/wk
H_l_6mv: 0.025714185328653116 mSv/wk
Total dose:  0.026525234414050608 mSv/wk
Dose limit:  0.1 mSv/wk
Thickness used: 90.9914868 cm = 35.82342 inches


In [61]:
# H_lt
d_leak = convert_to_meters(26,4.89957) # m

t_slant = convert_to_meters(2,8.13994) * 100 # cm

# TVL for 6 MV
tvl1_6MV = 34 # cm
tvle_6MV = 29 # cm

# B factors for 6 MV
B_IMW_6MV = B2(t_slant, tvl1_6MV, tvle_6MV)

print("H_lt")
print("--------------------------")
print(f"d_leak = {d_leak} m")
print(f"t_slant = {t_slant} cm")
print(f"B_IMW_6MW = {B_IMW_6MV}")
print("--------------------------")

H_Lt_6MV = find_Hlt(1435.5, 1, 0.25, t_slant, tvl1_6MV, tvle_6MV, d_leak)

H_lt_tot = H_Lt_6MV
print(f"H_Lt_6MV = {H_Lt_6MV} Sv/wk")



H_lt
--------------------------
d_leak = 8.049249078 m
t_slant = 81.63544759999999 cm
B_IMW_6MW = 0.0022771852317813677
--------------------------
H_Lt_6MV = 1.2613361159126881e-05 Sv/wk


# Manual test of H_leakage when the gantry is pointing towards Wall A. This checks secondary radiation. Thus, U = 1. This is not checking for H_Lt where we would use U = 0.25

In [62]:
d_leakage11 = convert_to_meters(23,8.71120) # m
print(f"Distance from target to point beyond door = {d_leakage11} m")
tttest = H_leak_unshielded(1435.5, d_leakage11) * B2(convert_to_meters(2,5.5) * 100, 34, 29)
print(f"H_leakage = {tttest} mSv/wk > (0.1 mSv/wk / 1)")
print("Does not pass if we use the perpendicular thickness. However, if we use oblique thickness, it does pass")
see_look = H_leak_unshielded(1435.5, d_leakage11) * B2(convert_to_meters(2,9.31297)*100, 34, 29)
print(f"see_look = {see_look} mSv/wk < (0.1 mSv/wk / 1) ")

Distance from target to point beyond door = 7.231664479999999 m
H_leakage = 0.10645041273223434 mSv/wk > (0.1 mSv/wk / 1)
Does not pass if we use the perpendicular thickness. However, if we use oblique thickness, it does pass
see_look = 0.04933825330354894 mSv/wk < (0.1 mSv/wk / 1) 


In [63]:
# H_ls

# Because of uniform gantry rotation, d_sec is the same for H_ls as H_ps
# area 1 is the same for H_ls as H_ps

alpha_1_ls = 9e-3 # albedo of A1, concrete, Co-60, 45 deg incidence, 0 deg reflection
# This is for 6 MV

H_ls_6MV = find_Hls(1435.5, 0.25, 1, alpha_1_ls, area_1, d_sec, dzz)
print("H_ls")
print(f"alpha_1_ls = {alpha_1_ls}")
print("--------------------------")
print(f"H_ls_6MV = {H_ls_6MV} Sv/wk")



H_ls
alpha_1_ls = 0.009
--------------------------
H_ls_6MV = 3.2454716694371212e-06 Sv/wk


In [64]:
H_G_6MV = 0.25 * H_s_6MV + H_ps_6MV + H_Lt_6MV + H_ls_6MV

H_total_6MV = (2.64 * H_G_6MV) * 1e6 # uSv/wk

print(f"H_G_6MV = {H_G_6MV} Sv/wk")
print("--------------------------")
print(f"H_total_6MV = {H_total_6MV} uSv/wk")
print("--------------------------")

H_G_6MV = 1.840403319185047e-05 Sv/wk
--------------------------
H_total_6MV = 48.586647626485245 uSv/wk
--------------------------
